In [89]:
# general purpose modules for handling data
import numpy as np
from numpy import array
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

# for loading telo data column containing individual
# telomere length values
from ast import literal_eval

# custom module for handling telomere length data
import telomere_methods_astros as telo_ma

In [84]:
import importlib
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
##### &nbsp; 

The dataframe containing our telomere length data already contains the means. However, we're interested in analyzing (graphing, stats) the individual telomere length measurements. To do so we must extract the values from the list, while keeping the values associated with their timepont/individual. We'll achieve this by 'exploding' the cell containing individual telomere length measurements as a list, into one row per measurement (5520 rows per sample, 184 telos per 30 metaphases per sample). 

As well, we're interested in defining and examining the prevalence of short/long telomeres; this is achieved by finding the values that divide the data into bottom 25% (short telos), middle 50%, and top 25% (long telos). More on that soon.

##### &nbsp; 
---

### Reading astronaut dataframe from file

In [90]:
astro_df = pd.read_csv('../excel data/All_astronauts_telomere_length_dataframe.csv')

# literal eval enables interpretation of individual telomere length values
# in the list 
astro_df['telo data'] = astro_df['telo data'].apply(lambda row: np.array(literal_eval(row)))

In [93]:
astro_df.head(4)

astro number  astro id timepoint flight status  \
0             1      5163     L-270    Pre-Flight   
1             1      5163     L-180    Pre-Flight   
2             1      5163      L-60    Pre-Flight   
3             1      5163      FD90    Mid-Flight   

                                           telo data  telo means  \
0  [46.9762779819579, 78.38289341797528, 105.1119...   87.672120   
1  [102.38890745071835, 107.3170731707317, 61.944...  101.077756   
2  [92.4323421316405, 144.537253591714, 135.59973...  128.599235   
3  [89.22485800200468, 116.6388239224858, 116.221...  101.183129   

                    Q1                           Q2-3                   Q4  
0  telos preF Q1 <0.25  telos preF Q2-3 >0.25 & <0.75  telos preF Q4 >0.75  
1  telos preF Q1 <0.25  telos preF Q2-3 >0.25 & <0.75  telos preF Q4 >0.75  
2  telos preF Q1 <0.25  telos preF Q2-3 >0.25 & <0.75  telos preF Q4 >0.75  
3  telos preF Q1 <0.25  telos preF Q2-3 >0.25 & <0.75  telos preF Q4 >0.75

### Exploding list containing individual telomere length measurements into rows

this action explodes the list into columns bearing each datapoint.
picture the list, which contains the individual values, expanding to the
right, up to 5520 measurements

importantly, the index #s per row still refer to the timepont/sample
so we can merge these columns back to the original astro_df
then we'll melt the columns into one, resulting in a lot of rows
where each has one individual telomere length measurement

In [8]:
explode_telos_raw = astro_df['telo data'].apply(pd.Series)

In [10]:
explode_telos_raw.head(2)

0           1           2           3          4           5     \
0   46.976278   78.382893  105.111928   37.236886  69.111260  101.620448   
1  102.388907  107.317073   61.944537  108.102239  94.854661   91.764116   

        6           7           8          9     ...        5510        5511  \
0  59.321751   61.443368   85.633144  80.838623  ...   58.453057  114.049449   
1  77.464083  114.316739  131.440027  89.876378  ...  106.698964   66.471767   

         5512       5513        5514        5515        5516        5517  \
0  135.967257  68.242566  101.553625  140.678249   82.308720   69.027731   
1   41.430003  89.909790  110.892082   83.578349  101.470097  163.431340   

         5518       5519  
0   54.376879  69.395256  
1  116.839292  44.136318  

[2 rows x 5520 columns]

In [13]:
exploded_telos_astro_df = (explode_telos_raw
    # merge 5520 columns with original dataframe
    .merge(astro_df, right_index = True, left_index = True)
                           
    # drop unnecessary columns
    .drop(['telo data', 'Q1', 'Q2-3', 'Q4'], axis = 1)
                           
    #specify which columns remain constant per indviidual telo 
    .melt(id_vars = ['astro number', 'astro id', 'timepoint', 'flight status', 'telo means'], value_name = "telo data exploded") 
    .drop("variable", axis = 1)
    .dropna())

exploded_telos_astro_df.shape

(436080, 6)

### Saving exploded individual telos for all astros dataframe to csv for later retrieval

In [15]:
copy_exploded_telos_astros_df = exploded_telos_astro_df

copy_exploded_telos_astros_df.to_csv('../excel data/exploded_telos_astros_df.csv', index = False)

### Defining short/mid/long telomeres w/ quartiles and counting per timepoint per astro

This function identifies the timepoint per astronaut most distal to spaceflight (L-270 or L-180) and identifies the individual telomere length values which divide the data into the bottom 25% (short telos), mid 50%, and top 25% (long telos). The function then counts how many telos reside in each quartile. Now we have a way to define short/long telomeres.

The function then applies those quartile cutoff values to subsequent datapoints and counts how many telomeres reside within each quartile. In doing so, we count the number of telomeres moving into/out of the quartiles, per timepoint per astronaut, and can quantitatively dicuss # of short/long telos.

In [142]:
quartiles_astro_df = telo_ma.make_quartiles_columns(astro_df)

In [141]:
%reload_ext autoreload

In [144]:
# for col in ['Q1', 'Q2-3', 'Q4']:
#     quartiles_astro_df[col] = quartiles_astro_df[col].astype('int64')

quartiles_astro_df['Q1'] = quartiles_astro_df['Q1'].astype('int64')
quartiles_astro_df['Q2-3'] = quartiles_astro_df['Q2-3'].astype('int64')
quartiles_astro_df['Q4'] = quartiles_astro_df['Q4'].astype('int64')

quartiles_astro_df[quartiles_astro_df['astro id'] == 2171]

astro number  astro id timepoint flight status  \
30             5      2171     L-180    Pre-Flight   
31             5      2171      L-60    Pre-Flight   
32             5      2171      FD45    Mid-Flight   
33             5      2171     FD260    Mid-Flight   
34             5      2171       R+5   Post-Flight   
35             5      2171     R+105   Post-Flight   
36             5      2171     R+180   Post-Flight   

                                            telo data  telo means    Q1  Q2-3  \
30  [79.20496894409938, 98.59627329192547, 110.509...  127.863014  1381  2759   
31  [224.4472049689441, 150.28571428571428, 119.00...  113.106425  1614  3235   
32  [21.664596273291924, 32.91925465838509, 44.459...  135.290127  1413  2112   
33  [214.22360248447205, 91.63975155279503, 219.81...  134.600340  1230  2663   
34  [75.1055900621118, 83.3913043478261, 95.801242...   87.765418  3009  2374   
35  [93.7888198757764, 114.3975155279503, 198.3478...  120.721879  1793  2600   
36  [150.8944099378882, 141.87577639751552, 214.59...  112.335962  1558  3379   

      Q4  
30  1380  
31   671  
32  1995  
33  1627  
34   137  
35  1127  
36   583

In [145]:
melted_quartiles_astro_df = pd.melt(
    quartiles_astro_df,
    id_vars=['astro number', 'astro id', 'timepoint', 'flight status', 'telo data', 'telo means'],
#     id_vars = [col for col in astro_df.columns if col != 'Q1' and col != 'Q2-3' and col != 'Q4'],
    var_name='relative Q',
    value_name='Q freq counts')

melted_quartiles_astro_df['Q freq counts'] = melted_quartiles_astro_df['Q freq counts'].astype('int64')
melted_quartiles_astro_df['astro id'] = melted_quartiles_astro_df['astro id'].astype('str')

In [146]:
melted_quartiles_astro_df[melted_quartiles_astro_df['astro id'] == '5163'].head(8)

astro number astro id timepoint flight status  \
0             1     5163     L-270    Pre-Flight   
1             1     5163     L-180    Pre-Flight   
2             1     5163      L-60    Pre-Flight   
3             1     5163      FD90    Mid-Flight   
4             1     5163     FD140    Mid-Flight   
5             1     5163       R+7   Post-Flight   
6             1     5163      R+60   Post-Flight   
7             1     5163     R+180   Post-Flight   

                                           telo data  telo means relative Q  \
0  [46.9762779819579, 78.38289341797528, 105.1119...   87.672120         Q1   
1  [102.38890745071835, 107.3170731707317, 61.944...  101.077756         Q1   
2  [92.4323421316405, 144.537253591714, 135.59973...  128.599235         Q1   
3  [89.22485800200468, 116.6388239224858, 116.221...  101.183129         Q1   
4  [161.89442031406617, 152.82325425993986, 223.3...  129.851970         Q1   
5  [88.57333778817241, 86.86936184430337, 84.9315...   82.169298         Q1   
6  [92.36551954560642, 75.97728032074842, 102.639...  100.116703         Q1   
7  [163.46475108586702, 157.11660541262947, 75.76...  115.403190         Q1   

   Q freq counts  
0           1384  
1            848  
2            259  
3            472  
4            203  
5           1876  
6            566  
7            936

### Saving melted-quartile-count all astros dataframe for later retrieval

In [147]:
copy_melted_quartiles_astro_df = melted_quartiles_astro_df
copy_melted_quartiles_astro_df['telo data'] = copy_melted_quartiles_astro_df['telo data'].apply(lambda row: row.tolist())


melted_quartiles_astro_df.to_csv('../excel data/melted_quartiles_astro_df.csv', index = False)